In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.9 MB/s eta 0:00:00


In [34]:
import pandas as pd
import numpy as np
import gurobipy
from gurobipy import Model, GRB, quicksum, tuplelist
from geopy.distance import great_circle
import math
import itertools
import random
import os
import shutil
from matplotlib import pyplot as plt
from copy import deepcopy

EAV_range = 200.0 #miles
avg_speed_NYC = 13.0 #mph
power_consumption_rate = 0.3 #kWh/mi
need_charging = 0.2 #SOC<20% range
charge_upto = 0.8 #charge upto 80%

max_pickup_time = 30.0 #minutes
max_pickup_dist = max_pickup_time*avg_speed_NYC/60 #miles
max_wait_time = 15.0
charging_power = 50.0 #kW

def trip_distance(long1, lat1, long2, lat2):
  GPS1 = (long1, lat1)
  GPS2 = (long2, lat2)
  return 1.4413*great_circle(GPS1, GPS2).miles + 0.1383 #miles

taxi = pd.read_csv('taxi_data.csv')
request = pd.read_csv('customer_trip_data.csv')

taxi_index = taxi.index.tolist()
request_index = request.index.tolist()
path = tuplelist(list(itertools.product(taxi_index, request_index)))
taxi = taxi.to_dict(orient="index")
request = request.to_dict(orient="index")

#pickup distance
pickup_distance = pd.DataFrame(index=taxi_index, columns=request_index)
pickup_distance = pickup_distance.to_dict(orient="index")
for each_taxi in taxi_index:
  for each_request in request_index:
    taxi_GPS = (taxi[each_taxi]['taxi_longitude'], taxi[each_taxi]['taxi_lattitude'])
    request_GPS = (request[each_request]['pickup_longitude'], request[each_request]['pickup_lattitude'])
    pickup_distance[each_taxi][each_request] = 1.4413*great_circle(taxi_GPS, request_GPS).miles + 0.1383 #miles

#pickup time
pickup_time_in_mins = deepcopy(pickup_distance)
for each_taxi in taxi_index:
  for each_request in request_index:
    pickup_time_in_mins[each_taxi][each_request] = int(math.ceil((pickup_distance[each_taxi][each_request]/avg_speed_NYC) * 60))
    
#pickup time + customer_waiting_time(mins)
delay_time_in_mins = deepcopy(pickup_time_in_mins)
for each_taxi in taxi_index:
  for each_request in request_index:
    delay_time_in_mins[each_taxi][each_request] = (pickup_time_in_mins[each_taxi][each_request]
            + request[each_request]['customer_waiting_time(mins)'])
    
#pickup distance + occupied distance dictionary
pickup_occupied_distance = deepcopy(pickup_distance)
for each_taxi in taxi_index:
  for each_request in request_index:
    pickup_occupied_distance[each_taxi][each_request] = pickup_distance[each_taxi][each_request] + request[each_request]['trip_distance']
            
#pickup distance + occupied distance + charging distance dictionary
pickup_occupied_CS_distance = deepcopy(pickup_distance)
for each_taxi in taxi_index:
  for each_request in request_index:
    pickup_occupied_CS_distance[each_taxi][each_request] = pickup_distance[each_taxi][each_request] + request[each_request]['trip_distance'] + request[each_request]['CS_distance']

#customer waiting time
eta = 1.0
customer_waiting_time = deepcopy(pickup_distance)
for each_taxi in taxi_index:
  for each_request in request_index:
    customer_waiting_time[each_taxi][each_request] = eta*request[each_request]['customer_waiting_time(mins)']

reward_function = deepcopy(pickup_distance)
for each_taxi in taxi_index:
  for each_request in request_index:
    reward_function[each_taxi][each_request] = max_pickup_time - pickup_time_in_mins[each_taxi][each_request] + customer_waiting_time[each_taxi][each_request]

model = Model()
model.setParam('OutputFlag', 0)

#variables
x = model.addVars(taxi_index, request_index, vtype=GRB.BINARY, name="x")
obj = quicksum(x[i,j]*reward_function[i][j] for i,j in path)
model.setObjective(obj, GRB.MAXIMIZE)

#add constraints
for each_taxi in taxi_index:
  for each_request in request_index:
    if (pickup_occupied_CS_distance[each_taxi][each_request]>(taxi[each_taxi]['soc']*EAV_range)):
      model.addConstr(x[each_taxi, each_request]==0) #taxi does not accept request if range is not enough
    if(pickup_time_in_mins[each_taxi][each_request] > max_pickup_time):
      model.addConstr(x[each_taxi, each_request] == 0) #taxi does not accept request if pickup time is more than max
    if(customer_waiting_time[each_taxi][each_request] > max_wait_time):
      model.addConstr(x[each_taxi, each_request]==0)

model.addConstrs(x.sum(i, '*')<=1 for i in taxi_index)
model.addConstrs(x.sum('*', j)<=1 for j in request_index)
    
#solve
model.optimize()
#print solution
# print(model.getVars()[0].varName[2])
for v in model.getVars():
  if v.x == 1.0:
    print(v.varName, v.x)
print('Obj:', model.objVal)

#analysis
# for each_taxi in taxi_index:
#   for each_request in request_index:
#     print((each_taxi, each_request, pickup_time_in_mins[each_taxi][each_request]))

x[0,3] 1.0
x[1,46] 1.0
x[2,7] 1.0
x[4,1] 1.0
x[5,30] 1.0
x[6,43] 1.0
x[7,11] 1.0
x[8,36] 1.0
x[9,34] 1.0
x[10,35] 1.0
x[11,16] 1.0
x[12,38] 1.0
x[13,19] 1.0
x[14,24] 1.0
x[15,10] 1.0
x[16,47] 1.0
x[17,40] 1.0
x[18,4] 1.0
x[19,23] 1.0
x[20,29] 1.0
x[21,13] 1.0
x[22,17] 1.0
x[23,8] 1.0
x[24,28] 1.0
x[25,27] 1.0
x[26,0] 1.0
x[27,9] 1.0
x[28,32] 1.0
x[29,49] 1.0
Obj: 1010.4000000000001


In [24]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle
import random
def trip_distance(long1, lat1, long2, lat2):
  GPS1 = (long1, lat1)
  GPS2 = (long2, lat2)
  return 1.4413*great_circle(GPS1, GPS2).miles + 0.1383 #miles

request = pd.read_csv('customer_trip_data.csv')
request_index = request.index.tolist()
cs_min = 0.2
cs_max = 1.2
cw_min = 1
cw_max = 20
for index in request_index:
  request.loc[index,'trip_distance'] = round(trip_distance(request['pickup_longitude'][index], request['pickup_lattitude'][index], 
                                                     request['dropoff_longitude'][index], request['dropoff_lattitude'][index]), 3)
  request.loc[index, 'CS_distance'] = round(cs_min + (cs_max - cs_min)*random.random(), 3)
  request.loc[index,'customer_waiting_time(mins)'] = round(cw_min + (cw_max - cw_min)*random.random(),1)
request.to_csv('customer_trip_data.csv', index = False)


taxi = pd.read_csv('taxi_data.csv')
taxi_index = taxi.index.tolist()
soc_min = 0.1
soc_max = 1.0

for index in taxi_index:
  taxi.loc[index, 'soc'] = round(soc_min + (soc_max - soc_min)*random.random(), 2)
taxi.to_csv('taxi_data.csv', index = False)